In [37]:
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest   
from scipy import stats

In [3]:
ga_hits = pd.read_csv('ga_hits.csv')
print("ga_hits Data:")
print(ga_hits.head())

print("ga_hits Info:")
print(ga_hits.info())
print(ga_hits.shape)

ga_hits Data:
                                  session_id    hit_date  hit_time  \
0  5639623078712724064.1640254056.1640254056  2021-12-23  597864.0   
1  7750352294969115059.1640271109.1640271109  2021-12-23  597331.0   
2   885342191847998240.1640235807.1640235807  2021-12-23  796252.0   
3   142526202120934167.1640211014.1640211014  2021-12-23  934292.0   
4  3450086108837475701.1640265078.1640265078  2021-12-23  768741.0   

   hit_number hit_type hit_referer  \
0          30    event         NaN   
1          41    event         NaN   
2          49    event         NaN   
3          46    event         NaN   
4          79    event         NaN   

                                       hit_page_path event_category  \
0  sberauto.com/cars?utm_source_initial=google&ut...           quiz   
1  sberauto.com/cars/fiat?city=1&city=18&rental_c...           quiz   
2  sberauto.com/cars/all/volkswagen/polo/e994838f...           quiz   
3  sberauto.com/cars?utm_source_initial=yandex&ut...

In [5]:
ga_sessions = pd.read_csv('ga_sessions.csv')

print("ga_sessions Data:")
print(ga_sessions.head())

print("ga_sessions Info:")
print(ga_sessions.info())
print(ga_sessions.shape)

C:\Users\Смбат\AppData\Local\Temp\ipykernel_16988\3736363291.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ga_sessions = pd.read_csv('ga_sessions.csv')


ga_sessions Data:
                                  session_id          client_id  visit_date  \
0  9055434745589932991.1637753792.1637753792  2108382700.163776  2021-11-24   
1   905544597018549464.1636867290.1636867290   210838531.163687  2021-11-14   
2  9055446045651783499.1640648526.1640648526  2108385331.164065  2021-12-28   
3  9055447046360770272.1622255328.1622255328  2108385564.162225  2021-05-29   
4  9055447046360770272.1622255345.1622255345  2108385564.162225  2021-05-29   

  visit_time  visit_number            utm_source utm_medium  \
0   14:36:32             1  ZpYIoDJMcFzVoPFsHGJL     banner   
1   08:21:30             1  MvfHsxITijuriZxsqZqt        cpm   
2   02:42:06             1  ZpYIoDJMcFzVoPFsHGJL     banner   
3   05:00:00             1  kjsLglQLzykiRbcDiGcD        cpc   
4   05:00:00             2  kjsLglQLzykiRbcDiGcD        cpc   

           utm_campaign         utm_adcontent           utm_keyword  \
0  LEoPHuyFvzoNfnzGgfcd  vCIpmpaGBnIQhyYNkXqp  puhZPIYqKX

In [7]:
missing_hits = ga_hits.isnull().sum()
print("Пропуски в ga_hits:", missing_hits[missing_hits > 0])

Пропуски в ga_hits: hit_time        9160322
hit_referer     6274804
event_label     3760184
event_value    15726470
dtype: int64


In [9]:
missing_sessions = ga_sessions.isnull().sum()
print("Пропуски в ga_sessions:", missing_sessions[missing_sessions > 0])

Пропуски в ga_sessions: utm_source            97
utm_campaign      219603
utm_adcontent     335615
utm_keyword      1082061
device_os        1070138
device_brand      367178
device_model     1843704
dtype: int64


In [11]:
duplicates_hits = ga_hits.duplicated().sum()
print("Количество дубликатов в ga_hits:", duplicates_hits)

Количество дубликатов в ga_hits: 0


In [13]:
duplicates_sessions = ga_sessions.duplicated().sum()
print("Количество дубликатов в ga_sessions:", duplicates_sessions)

Количество дубликатов в ga_sessions: 0


In [13]:
threshold_hits = int(0.4 * len(ga_hits)) 
ga_hits_cleaned = ga_hits.dropna(axis=1, thresh=threshold_hits)
print("Остальные столбцы в ga_hits:", ga_hits_cleaned.columns)

Остальные столбцы в ga_hits: Index(['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type',
       'hit_referer', 'hit_page_path', 'event_category', 'event_action',
       'event_label'],
      dtype='object')


In [15]:
threshold_sessions = int(0.4 * len(ga_sessions))  
ga_sessions_cleaned = ga_sessions.dropna(axis=1, thresh=threshold_sessions)
print("Остальные столбцы в ga_sessions:", ga_sessions_cleaned.columns)

Остальные столбцы в ga_sessions: Index(['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'utm_keyword', 'device_category', 'device_os', 'device_brand',
       'device_screen_resolution', 'device_browser', 'geo_country',
       'geo_city'],
      dtype='object')


In [17]:
merged_data = pd.merge(ga_hits, ga_sessions, on='session_id', how='inner')
print("Объединенный датафрейм:", merged_data.head())

Объединенный датафрейм:                                   session_id    hit_date  hit_time  \
0  5639623078712724064.1640254056.1640254056  2021-12-23  597864.0   
1  7750352294969115059.1640271109.1640271109  2021-12-23  597331.0   
2   885342191847998240.1640235807.1640235807  2021-12-23  796252.0   
3   142526202120934167.1640211014.1640211014  2021-12-23  934292.0   
4  3450086108837475701.1640265078.1640265078  2021-12-23  768741.0   

   hit_number hit_type hit_referer  \
0          30    event         NaN   
1          41    event         NaN   
2          49    event         NaN   
3          46    event         NaN   
4          79    event         NaN   

                                       hit_page_path event_category  \
0  sberauto.com/cars?utm_source_initial=google&ut...           quiz   
1  sberauto.com/cars/fiat?city=1&city=18&rental_c...           quiz   
2  sberauto.com/cars/all/volkswagen/polo/e994838f...           quiz   
3  sberauto.com/cars?utm_source_initial=ya

In [19]:
merged_data = merged_data.drop_duplicates()
for column in merged_data.columns:
    if merged_data[column].dtype == 'object':
        merged_data[column].fillna('unknown', inplace=True)
    else:
        merged_data[column].fillna(merged_data[column].mean(), inplace=True)

print("Очистка данных завершена. Информация о датафрейме:", merged_data.info())

C:\Users\Смбат\AppData\Local\Temp\ipykernel_16988\1507269359.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data[column].fillna('unknown', inplace=True)
C:\Users\Смбат\AppData\Local\Temp\ipykernel_16988\1507269359.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15685219 entries, 0 to 15685218
Data columns (total 28 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   session_id                object 
 1   hit_date                  object 
 2   hit_time                  float64
 3   hit_number                int64  
 4   hit_type                  object 
 5   hit_referer               object 
 6   hit_page_path             object 
 7   event_category            object 
 8   event_action              object 
 9   event_label               object 
 10  event_value               float64
 11  client_id                 object 
 12  visit_date                object 
 13  visit_time                object 
 14  visit_number              int64  
 15  utm_source                object 
 16  utm_medium                object 
 17  utm_campaign              object 
 18  utm_adcontent             object 
 19  utm_keyword               object 
 20  device_category       

In [21]:
print(merged_data.columns)
merged_data['is_organic'] = merged_data['utm_source'].apply(lambda x: 1 if x == 'organic' else 0)
merged_data['is_mobile'] = merged_data['device_category'].apply(lambda x: 1 if x == 'mobile' else 0)
represented_categories = ['category1', 'category2']  
merged_data['is_represented'] = merged_data['event_category'].apply(lambda x: 1 if x in represented_categories else 0)
social_sources = ['facebook', 'twitter', 'instagram']  
merged_data['is_social'] = merged_data['utm_source'].apply(lambda x: 1 if x in social_sources else 0)
print(merged_data.info())

Index(['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type',
       'hit_referer', 'hit_page_path', 'event_category', 'event_action',
       'event_label', 'event_value', 'client_id', 'visit_date', 'visit_time',
       'visit_number', 'utm_source', 'utm_medium', 'utm_campaign',
       'utm_adcontent', 'utm_keyword', 'device_category', 'device_os',
       'device_brand', 'device_model', 'device_screen_resolution',
       'device_browser', 'geo_country', 'geo_city'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15685219 entries, 0 to 15685218
Data columns (total 32 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   session_id                object 
 1   hit_date                  object 
 2   hit_time                  float64
 3   hit_number                int64  
 4   hit_type                  object 
 5   hit_referer               object 
 6   hit_page_path             object 
 7   event_category            o

In [23]:
click_counts = merged_data.groupby('client_id').size().reset_index(name='click_count')
click_threshold = 10
print(f"Порог для аномальных кликов: {click_threshold}")
valid_clients = click_counts[click_counts['click_count'] <= click_threshold]['client_id']
cleaned_data = merged_data[merged_data['client_id'].isin(valid_clients)]
print("Размер очищенного датафрейма:", cleaned_data.shape)

Порог для аномальных кликов: 10
Размер очищенного датафрейма: (2968707, 32)


In [75]:
print(merged_data.columns)


Index(['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type',
       'hit_referer', 'hit_page_path', 'event_category', 'event_action',
       'event_label', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'utm_keyword', 'device_category', 'device_os', 'device_brand',
       'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city',
       'is_organic', 'is_mobile', 'is_represented', 'is_social'],
      dtype='object')


In [ ]:
Гипотезы:
Нулевая гипотеза (H0): Конверсионные ставки для группы A и группы B равны. (Нет эффекта от изменений.)
Альтернативная гипотеза (H1): Конверсионные ставки для группы A и группы B различны. (Есть эффект от изменений.)

In [ ]:
Выбор критерия:
Поскольку данные распределенны не нормально, то я выбрал U-теста Манна-Уитни.

In [101]:
 # 1. Выбор количественного признака для анализа
# Например, используем 'hit_number'
data_to_test = merged_data['visit_number'].dropna()  # Удаляем пропуски

# 2. Проверка на нормальность распределения
shapiro_test = stats.shapiro(data_to_test)
print('Статистика теста Шапиро-Уилка:', shapiro_test.statistic)
print('p-значение:', shapiro_test.pvalue)

if shapiro_test.pvalue > 0.05:
    print("Данные нормально распределены (не отвергаем H0)")

Статистика теста Шапиро-Уилка: 0.10057269988677486
p-значение: 3.670896692861485e-239


D:\Program Files\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 15685219.
  res = hypotest_fun_out(*samples, **kwds)


In [124]:
# Определение групп
group_a = merged_data[merged_data['device_category'] == 'mobile']['hit_number']
group_b = merged_data[merged_data['device_category'] == 'desktop']['hit_number']

# Проведение U-теста Манна-Уитни
u_statistic, p_value = stats.mannwhitneyu(group_a, group_b)

print('U-статистика:', u_statistic)
print('p-значение:', p_value)

# Интерпретация результатов
alpha = 0.05  # уровень значимости
if p_value < alpha:
    print("Существует статистически значимое различие между группами.")
else:
    print("Нет статистически значимого различия между группами.")

U-статистика: 19645415631375.0
p-значение: 0.0
Существует статистически значимое различие между группами.


In [ ]:
На основании полученных данных из U-теста Манна-Уитни можно сделать следующие выводы относительно гипотез:
1. Нулевая гипотеза (H0): Конверсионные ставки для группы A и группы B равны. (Нет эффекта от изменений.)
2. Альтернативная гипотеза (H1): Конверсионные ставки для группы A и группы B различны. (Есть эффект от изменений.)
Выводы
Поскольку p-значение равно 0.0, это означает, что мы можем отклонить нулевую гипотезу (H0) и принять альтернативную гипотезу (H1). 
Таким образом:
Нулевая гипотеза (H0) отклоняется.
Альтернативная гипотеза (H1) принимается.
Заключение
Существует статистически значимое различие между конверсионными ставками для группы A и группы B. Это подтверждает, что изменения, 
которые вы внедрили, оказали влияние на конверсионные ставки.

In [1]:
for column in columns_to_fill:
    most_frequent_value = merged_data[column].mode()[0]  
    merged_data[column] = merged_data[column].fillna(most_frequent_value)

NameError: name 'columns_to_fill' is not defined

In [41]:
missing_values = merged_data.isnull().sum()
missing_percentage = (missing_values / len(merged_data)) * 100
threshold = 60
columns_with_many_missing = missing_percentage[missing_percentage > threshold]
print("Количество пропусков в каждом столбце:")
print(missing_values)
print("\nДоля пропусков в каждом столбце:")
print(missing_percentage)
print(f"\nСтолбцы с более чем {threshold}% пропусков:")
print(columns_with_many_missing)

Количество пропусков в каждом столбце:
session_id                  0
hit_date                    0
hit_time                    0
hit_number                  0
hit_type                    0
hit_referer                 0
hit_page_path               0
event_category              0
event_action                0
event_label                 0
client_id                   0
visit_date                  0
visit_time                  0
visit_number                0
utm_source                  0
utm_medium                  0
utm_campaign                0
utm_adcontent               0
utm_keyword                 0
device_category             0
device_os                   0
device_brand                0
device_screen_resolution    0
device_browser              0
geo_country                 0
geo_city                    0
is_organic                  0
is_mobile                   0
is_represented              0
is_social                   0
dtype: int64

Доля пропусков в каждом столбце:
session_id    

In [47]:
columns_to_fill = ['hit_time', 'hit_referer', 'event_label', 
                   'utm_source', 'utm_campaign', 
                   'utm_adcontent', 'utm_keyword', 
                   'device_os', 'device_brand']
print("Количество пропусков после заполнения:")
print(merged_data[columns_to_fill].isnull().sum())

Количество пропусков после заполнения:
hit_time         0
hit_referer      0
event_label      0
utm_source       0
utm_campaign     0
utm_adcontent    0
utm_keyword      0
device_os        0
device_brand     0
dtype: int64


In [89]:
missing_values = merged_data.isnull().sum()
print("Количество пустых значений в каждом столбце:")
print(missing_values[missing_values > 0])  

Количество пустых значений в каждом столбце:
Series([], dtype: int64)
